# Lab 9: A Recurrence Relation \& The Master Theorem
## Data Structures & Algorithms
10&11/04/2025

# Remember MergeSort? 

## A Divide-&-Conquer refresher
Divide-and-conquer algorithms are a class of algorithms that solve a problem by:
1. **Divide**: Breaking the problem into smaller, more manageable subproblems.
2. **Conquer**: Solving the subproblems recursively.
3. **Combine**: Merging the solutions of the subproblems to solve the original problem.

## MergeSort
1. divides the array into two halves, 
2. sorts each half recursively, 
3. and then merges the sorted halves.

<div>
   <img src="images/mergesort_viz.png" width="500px" title="mergesort visualisation">
</div>

## In code:

In [6]:
def merge_sort(arr):
    """
    Merge sort 
    
    Parameters
    ----------
    arr : a list of number

    Returns
    ----------
    The list sorted in ascending order
    """
    arr_temp = list(arr)
    n = len(arr_temp)    
    
    if n > 1: 
        # STEP 1: DIVIDE
        # Divide the list into two smaller ones
        # The middle of the list
        mid = n // 2 # using floor division (a.k.a integer division)
        # The left sublist
        arr_temp_left = arr_temp[:mid] 
        # The right sublist
        arr_temp_right = arr_temp[mid:]

        # STEP 2: RECURSIVE CALL (UNTIL N=1)
        # Recursively call merge_sort to sort the two smaller lists
        arr_temp_left = merge_sort(arr_temp_left)
        arr_temp_right = merge_sort(arr_temp_right)
        
        # STEP 3: MERGE  
        # Merge the two sorted smaller lists
        i = j = k = 0
        n_left, n_right = len(arr_temp_left), len(arr_temp_right)
          
        while i < n_left and j < n_right: # this while statement says to keep going through the two lists until one of them is exhausted
            if arr_temp_left[i] < arr_temp_right[j]: 
                arr_temp[k] = arr_temp_left[i] 
                i += 1
            else: 
                arr_temp[k] = arr_temp_right[j] 
                j += 1
            k += 1
          
        # If there are elements in arr_temp_left that have not been visited 
        while i < n_left: 
            arr_temp[k] = arr_temp_left[i] 
            i += 1
            k += 1
 
        # If there are elements in arr_temp_right that have not been visited 
        while j < n_right: 
            arr_temp[k] = arr_temp_right[j] 
            j += 1
            k += 1
            
    return arr_temp

## Compared to brute force BubbleSort

Remember, BubbleSort (naive / brute force) was $O(n^2)$; whereas MergeSort (efficient) was $O(n\log(n))$.

In [7]:
def bubble_sort(arr):
    """
    Bubble sort
    
    Parameters
    ----------
    arr : a list of number

    Returns
    ----------
    The list sorted in ascending order
    """
    n = len(arr)
    
    for i in range(n):
        for j in range(0, n-i-1):
            if arr[j] > arr[j+1]:
                arr[j], arr[j+1] = arr[j+1], arr[j]
                
    return arr

# execute:

import random
import time

random.seed(42)
long_arr = [random.randint(0, 10_000) for _ in range(5000)]

# let's time bubble sort
start_bubble = time.time()
bubble_sort_brute_force(long_arr)
end_bubble = time.time()
bubble_time = end_bubble - start_bubble

# ...and merge sort
start_merge = time.time()
merge_sort(long_arr)
end_merge = time.time()
merge_time = end_merge - start_merge

# Print results
print(f"Bubble Sort Time: {bubble_time:.4f} seconds")
print(f"Merge Sort Time:  {merge_time:.4f} seconds")
print(f"Merge sort is {bubble_time / merge_time:.2f} times faster than bubble sort")

Bubble Sort Time: 0.6061 seconds
Merge Sort Time:  0.0038 seconds
Merge sort is 158.67 times faster than bubble sort


# What's New?

So all of that should be familiar from lab 8 (if not, go back through lab 8!), but we didn't spend long thinking about how to formally conceptualise and analyse why and how divide-and-conquer algorithms are more efficient than their brute force counterparts. As an important class of algorithm, there's significant work gone into thinking about how, why, and under what conditions Divide-and-Conque algorithms are efficient.

First, let's *intuitively* understand why MergeSort specifically is $O(n \log n)$. 

## Intuition behind MergeSort

MergeSort is a recursive algorithm (it calls itself); the most intuitive way to think about it is to 'unroll' the recurrence and look at patterns in the first few levels:

<div>
   <img src="images/screenshot_rectree_mergesort.png" width="500px">
</div>

At a high level, some points that should be immediately obvious:
- at the top leel, the amount of 'work' (i.e. items to sort) MergeSort has to do is a function of $n$ (the size of the input array),
- at each level of recursion, the number of subproblems we're dealing with doubles,
- at each level of recursion, the size of each subproblem halves,
- the total amount of 'work' thus remains constant: at $n$.

So that's one insight - at all levels the amount of 'work' to be done remains $n$. 

Now our problem is that if we were to do all that $n$ amount of work at the top level (without any dividing nonsense), we already know (from BubbleSort) that is $O(n^2)$. But if we Divide-and-Conquer, we can increase the efficiency of getting the same amount of work done. 

## Thinking through MergeSort in terms of base cases vs levels (an unrolled recurrence)

First off, at all levels, the division operation is $c$ ($O(1)$) so we can effectively ignore it and focus on the merge and recurion elements! (as a recursive algorithm, in MergeSort they are jointly handled as part of recursive workflow).

This takes us down to the base-case level (when as a single-valued array, they have been sorted), and in the combine step (where they are combined *in order*). We are therefore interested in how the number of base cases, and the number of combine ooerations scale wrt input size. For an $n$ lengthed array:
- The **number of base cases** is also $n$. This gives us the $n$ of the $O(n \log n)$.
- The **number of combine steps** is given by the *depth* of the recursion (i.e. how many steps does it have to take to recombine all the base cases into a single array again). As we are halving the number of subproblems at each step, this is given by $\log n$ (the base 2 inverse exponential). This gives us the $\log n$ of the $O(n \log n)$.

NB: We decomposed MergeSort in the above way because it is the most intuitive. However, it is isomorphic to (i.e., structurally equivalent to) the following decomposition, which we will use when deriving the recurrence relation:
- **number of combine step**: $n$, as the number of comparisons we have to do between values is $n$ (every value has to be compared at one point during our combining steps)
- **nummber of levels of recursion**: $\log n$.

#### More formally:

1. **Top Level (Entire Array of Size $n$)**
   - This is self-evident. 
   - We also make two recursive calls to sort the left and right halves.
2. **Second Level (Two Subproblems of Size $n/2$)**
   - Each of the two halves is of size $n/2$.
   - Each half takes at most $O(n/2)$ time for merging.
   - Since there are two such halves, the total time at this level is still $O(n)$.
3. **Third Level (Four Subproblems of Size $n/4$)**
   - Each subproblem is now $n/4$ in size.
   - Each takes at most $O(n/4)$ time for merging.
   - Since there are four such subproblems, the total time at this level remains $O(n)$.
- **Generalizing to Deeper Levels**
   - At each level of recursion, the total merging time remains $O(n)$ because there are always enough subproblems to sum up to $n$.
   - The recursion continues until we reach base cases (single-element arrays).

#### So at level $j$: 
1. Number of Subproblems at Level $j$
   - At each level of recursion, the number of subproblems **doubles** compared to the previous level.
   - We start with **1 subproblem** (the entire array), after $j$ levels, there are: $2^j \text{ subproblems}$

2. Size of Each Subproblem at Level $j$
   - The array is divided in half at every level.
   - After $j$ levels, each subproblem is of size: $\frac{n}{2^j}$:
      - after **one split**, the size is $ n/2 $, 
      - after **two splits**, it's $ n/4 $, 
      - and so on.

3. Total Work Done at Level $j$
   - As before, the division operation is done in constant time, so it drops out. This leaves us just with merging and recursion.
   - Each of the $ 2^j $ subproblems requires at most $ O(n/2^j) $ time for merging.
   - Since there are $ 2^j $ such subproblems, the total work done at level $j$ is: $2^j \cdot \frac{n}{2^j} = n$

**The key point to get here is that the total work remains $ O(n) $ at **every** level!**

4. Summing Over All Levels
   - The recursion continues until we reach base cases where the subproblem size is **1**.
   - The number of levels required to reduce $ n $ down to **1** is the number of times we can halve $ n $:
   
   $$\log_2 n$$
   
   - Since we do **$ O(n) $ work at each level**, and there are $ O(\log n) $ levels, the total time complexity is:
   $$O(n \log n)$$

# Fomalising that intuition

There are two conceptual tools that help us to make the jump from thinking about MergeSort speceifically, to generalising / abstracting to all Divide-and-conquer algorithms:
1. A Recurrence Relation
2. The Master Theorem

# A Recurrence Relation

*"A recurrence relation is an equation that defines a sequence based on previous terms of the sequence. It expresses the value of a function (often representing the cost or size of a process) at size $n$, like $T(n)$, in terms of its values at smaller inputs, like  $T(n−1)$ or $T(n/2)$."*

### Thinking through MergeSort in terms of Divide-&-Conquer recursion

First, let's say $T(n)$ is the worst-case running time of the algorithm for an input of size $n$.

For MergeSort: $T(n)$ can be decomposed into 3 separate processes at each step:
   1. **Divide**: subdividing the problem into 2 pieces (of size $n/2$). 
      - *NB: the division part involves indexing an array, which is done in in constant-time $O(1)$, so it drops out*
      - so we now have 2 problems.
   2. **Conquer**: we then know that the algorithm will spend $T(n/2)$ on each sub-problem. 
      - so we now have 2 problems, each of size $T(n/2)$.
   3. **Combine**: plus *some amount of time* combining the subproblems. 
      - the combine step involves comparing each $i$ and $j$ index: this scales linearly.
      - so, that overhead will scale linearly with input size ->  $O(n)$
      - so we now have 2 problems, each of size $T(n/2)$ and *some overhead that scales linearly*.



The running time of MergeSort algorithm therefore satisfies the following recurrence relation:

<div>
   <img src="images/screenshot_mergesort.png" width="300px">
</div>

There are different ways of solving such a recurrence relation (i.e. make $T$ only appear on the left-hand side of the inequality). One would be to 'unroll' the recurrence as we did above, another is to use the Master Theorem.

# Master Theorem
2
The Master Theorem provides a systematic way to analyze the runtime of divide-and-conquer algorithms by expressing their recurrence relation in a general form:

$$
T(n) = aT\left(\frac{n}{b}\right) + f(n)
$$

where:
- $ a $ is the number of recursive subproblems created at each step.
- $ b $ is the factor by which the problem size shrinks at each level.
- $ c $ is the cost of the merging or combining step.

This allows us to derive 3 important 'properties' of any Divide-and-conquery algorithm:
- $1 + \log_b n$ is the number of levels (the **depth**),
  - $\log_b$ is the number of recursive steps
  - +1 is given by the top layer's existence
- $\frac{n}{b^j}$ is the **size** of subproblems at level $j$,
- $a^j$ is the **number** of subproblems at level $j$.

These 3 properties will be all we need to know of any divide-and-conquer algorithm to be able to analyse it using the Master Theorem.

### Applying Master Theorem to MergeSort

  <div>
   <img src="images/mergesort_viz.png" width="500px" title="mergesort visualisation">
  </div>

The running time of MergeSort satisfies the recurrence:


$$T(n) = 2 \, T\left(\frac{n}{2}\right) + cn$$

where:
- $2$ subproblems are solved,
- each subproblem has size $ \frac{n}{2} $,
- and $cn$ is the time to merge the two sorted halves.

This gives the following properties for MergeSort:
- $1 + \log_2 n$ is the number of levels (the **depth**),e
- $\frac{n}{2^j}$ is the **size** of subproblems at level $j$,
- $2^j$ is the **number** of subproblems at level $j$.

We intuited all this previously!

## Why We Care About the Master Theorem: General Form of Running Time

At first this just complicated the more intuitive unrolling idea. But by formalising the Running Time in a more abstracted way, the Master Theorem lets us analyse generic Divide-and-Conquer problems in some interesting way.

Following a pattern similar to the **recursion tree analysis**, we sum the work done **at each level** of recursion: given by the relationship between the 'properties' of any divide-and-conquer algorithm. Those properites were:
  1. the depth of the recursion, 
  2. the scaling of the subproblems at each level (how much are they reduced), 
  2. and the number of subproblems produced at each level.

Formally:
- The recursion **depth** is $ \log_b n $ (since the problem size reduces by a factor of $ b $ at each level).
- At each level, the work done depends on:
  1. **The number of subproblems** at that level. ($a^i$)
  2. **The size of each subproblem** and the time required to process it. ($\frac{n}{b^i}$)

Putting it all together:
These factors determine the **recurrence ratio** $ r $, which helps 1) classify the running time into one of three cases; 2) derives the total running time.

$$
r = \text{recurrence ratio}
$$

## Classifying Time Complexity with the Master Theorem's Recurrence Ratio
Formally, we can define the following:

$$
r = \frac{a}{b^c}
$$

This makes intuitive sense if you consider:
- $ a $ # of recursive subproblems created at each step.
- $ b $ factor by which problem size shrinks at each level.
- $ c $ cost of merging / combining.

The total running time follows:

$$
T(n) = n^c \sum_{i=0}^{\log_b n} r^i
$$

which leads to the following cases:

$$
T(n) =
\begin{cases} 
O(n^c) & \text{if } r < 1 \quad (c > \log_b a) \quad \text{(Root-dominated)} \\
O(n^c \log n) & \text{if } r = 1 \quad (c = \log_b a) \quad \text{(Balanced)} \\
O(n^{\log_b a}) & \text{if } r > 1 \quad (c < \log_b a) \quad \text{(Leaf-dominated)}
\end{cases}
$$

### Root-Dominated, Balanced, and Leaf-Dominated Cases
The Master Theorem helps determine whether an algorithm's time complexity is:
- **Root-dominated**: Work is mainly done at the top level (combining step).
- **Balanced**: Work is evenly distributed across levels.
- **Leaf-dominated**: Work is mainly done at the leaves (smallest subproblems).


## ... getting back to MergeSort
We know that for MergeSort:
1. two sub-problems are created at evvery step
  - $ a = 2$
2. the problem is divided into **subproblems** of size $ n/2 $
  - $ b = 2 $
3. **combining step** takes linear time ($ c = 1 $)
  - $ c = 1$ 

This gives us $r = 1$: it is a 'balanced\linear' divide-and-conquer algorith (it produces two subproblems at each level, but each subproblem's size is halved). Thus its runtime is $O(n \log n)$

This makes intuitive sense. We already looked at why it is $O(n \log n)$, and we also said in lab 8 that the 'magic' happens as MergeSort merges the lists into sorted lists or increasing size; this is done at the every combining stage of every recursive step. This validates what we already knew, but as a formalised theorem, the Master Theorem also gives us the tools to generalise this to the whole class of algorithms.

## Extrapolating Out  
As with MergeSort, consider an algorithm where:
1. The **combining step** takes linear time ($ c = 1 $).
2. The problem is divided into **subproblems** of size $ n/2 $ (i.e., $ b = 2 $).

Thus, we compute:

$$
r = \frac{a}{2}
$$

For different values of $ a $, we get different running times:

| **Number of Subproblems ($ a $)** | **Running Time** | **Classification** |
|--------------|----------------|------------------|
| $ a = 1 $ | $ O(n) $ | **Root-dominated** (Cost dominated by the top level) |
| $ a = 2 $ | $ O(n \log n) $ | **Balanced** (Merge Sort case) |
| $ a > 2 $ | $ O(n^{\log_2 a}) $ | **Leaf-dominated** (Cost dominated by the leaves) |


### Intuition Behind These Cases
- **When $ a = 1 $:** 
  - Each recursive call reduces the problem size, but since there's only one subproblem at each step, the total work remains **linear**.
  - The **top-level work (combining step)** dominates the complexity.
  <div>
   <img src="images/screenshot_rectree_a1.png" width="200px">
  </div>  
  
- **When $ a = 2 $ (Merge Sort case):** 
  - Each level does roughly the same amount of work.
  - The recursion **depth is $ \log n $**, and each level takes $ O(n) $, so the total time is $ O(n \log n) $.
  - Work is **evenly distributed across levels**.
  <div>
   <img src="images/screenshot_rectree_mergesort.png" width="500px">
  </div>

- **When $ a > 2 $:**
  - The number of subproblems grows **faster than the problem size shrinks**, leading to an increasing workload at the lower levels.
  - The **leaves of the recursion tree** dominate the total complexity.
  <div>
   <img src="images/screenshot_rectree_a3.png" width="500px">
  </div>

(All visualisations of recursive trees are taken from the Kleinberg textbook, which also has great further explanations in it!).


## Exercises

### Exercise on Master Theorem

Use the Master Theorem to give time complexity of Sort-and-Count and say whether it is root-dominated, leaf-dominated or balanced.

```
Sort-and-Count(L)
If the list has one element:
    there are no inversions
Else
    Divide the list into two halves:
        A contains the first ⌈n/2⌉ elements
        B contains the remaining ⌊n/2⌋ elements
    (rA, A) = Sort-and-Count(A)
    (rB, B) = Sort-and-Count(B)
    (r,L) = Merge-and-Count(A,B)
Endif
Return r =rA +rB +r, and the sorted list L

The recurrence relation is $T(n) = 2 T(\frac{n}{2}) + O(n)$. 

This is due to the fact that the `merge_and_count` algorithm you wrote in Exercise 5 of lab 8 has time complexity $O(n)$ and because we have two subproblems at each level, each of which is always half the size of the size at the previous level.

We therefore have $a=2$, $b=2$, and $c=1$, so $r=\frac{a}{b^c}= 1$, which - according to the Master Theorem - means that we have time complexity $O(n \log n)$ and the algorithm is **balanced**.